# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [5]:
# 2) seu código aqui
valores_ausentes = sinasc.isnull().sum()
valores_ausentes

#CODESTAB      115
#munResLat       1
#munResLon       1
#munResAlt       1
#munResArea      1

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [13]:
# 3) seu código aqui
#criando base com menos campos
base_reduzida = sinasc[['LOCNASC','IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO','GESTACAO','GRAVIDEZ','CONSULTAS','APGAR5']]

base_reduzida.isnull().sum()

#LOCNASC          0
#IDADEMAE         0
#CONSULTAS        0

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [16]:
# 4) seu código aqui

# Contagem de valores ausentes antes da remoção
valores_ausentes_antes = base_reduzida['APGAR5'].isnull().sum()

# Remover registros com Apgar5 não preenchido
base_reduzida = base_reduzida.dropna(subset=['APGAR5'])

# Contagem de valores ausentes após a remoção
valores_ausentes_depois = base_reduzida['APGAR5'].isnull().sum()

# Contagem de linhas após remoção
num_linhas_apos_remocao = len(base_reduzida)

print(f"Número de linhas antes da remoção: {len(base_reduzida) + valores_ausentes_antes}")
print(f"Número de valores ausentes antes da remoção: {valores_ausentes_antes}")
print(f"Número de linhas após a remoção: {num_linhas_apos_remocao}")
print(f"Número de valores ausentes após a remoção: {valores_ausentes_depois}")

#Número de linhas antes da remoção: 26925
#Número de valores ausentes antes da remoção: 0
#Número de linhas após a remoção: 26925
#Número de valores ausentes após a remoção: 0



Número de linhas antes da remoção: 26925
Número de valores ausentes antes da remoção: 0
Número de linhas após a remoção: 26925
Número de valores ausentes após a remoção: 0


In [23]:
# 5) seu código aqui
import numpy as np

base_reduzida['ESTCIVMAE'].replace(9, np.nan, inplace=True)
base_reduzida['CONSULTAS'].replace(9, np.nan, inplace=True)

In [24]:
# 6) Seu código aqui
base_reduzida['QTDFILVIVO'].replace(np.nan, 0, inplace=True)


In [38]:
# 7) seu código aqui
#base_reduzida['CONSULTAS'].value_counts()
#base_reduzida['GRAVIDEZ'].value_counts()
base_reduzida['QTDFILVIVO'].value_counts()

base_reduzida.loc[base_reduzida['QTDFILVIVO'] >= 6, 'QTDFILVIVO'] = '6 OU MAIS'

#Agrupando a quantidade de filhos que deixava o número de categorias mais dispersos, uma vez que a 
#quantidade de filhos não tem limite mas vai ter uma "calda" muito grande em uma distribuição normal
#reduzindo essa quantidade de categorias com poucos dados, é uma forma de "normalizar" os dados

QTDFILVIVO
0.0          10871
1.0           9361
2.0           4454
3.0           1362
4.0            475
5.0            212
6 OU MAIS      190
Name: count, dtype: int64

In [41]:
# 8) seu código aqui
intervalos = [-1, 3, 5, 7, 10]
categorias = ['Asfixia Severa', 'Asfixia Moderada', 'Asfixia Leve', 'Normal']

# Criar a nova coluna categorizando o Apgar
base_reduzida['Apgar_Categorizado'] = pd.cut(base_reduzida['APGAR5'], bins=intervalos, labels=categorias)

# Calcular as frequências da nova categorização
base_reduzida['Apgar_Categorizado'].value_counts()




Apgar_Categorizado
Normal              26463
Asfixia Leve          320
Asfixia Severa         74
Asfixia Moderada       68
Name: count, dtype: int64

In [44]:
# 9) seu código aqui
def rename_columns_to_snake_case(dataframe):
    columns = dataframe.columns.tolist()
    
    # Criar um dicionário para mapear os nomes das colunas originais para os nomes em snake case
    new_column_names = {col: col.lower().replace(' ', '_') for col in columns}
    
    # Renomear as colunas do DataFrame
    dataframe.rename(columns=new_column_names, inplace=True)
    
    return dataframe

rename_columns_to_snake_case(base_reduzida)

base_reduzida

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar_categorizado
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0,Normal
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4.0,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4.0,9.0,Normal
